In [1]:
# -*- coding: utf-8 -*-

In [2]:
import os
import pandas as pd 
from glob import glob

In [3]:
import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

In [4]:
import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

In [5]:
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

In [6]:
import chat_bot as cb
import Database_Handler as dh

In [7]:
ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1464079, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


## 언론사 리스트

> stopwords를 위해서

In [12]:
pressList = pd.read_excel('./data/presslist.xlsx')

with(open('./data/newspress.txt','w',encoding='utf-8')) as f:
    f.write('\n'.join(set(pressList['언론사'].tolist())))
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwordList = Stopwords('./data/newspress.txt')

# 자연어 처리

## TextRank 

In [13]:
swords =['이제','너무','중인' ,'만큼', '마찬가지', '꼬집었', '기자','습니다','어요','이렇게', "아", "휴", 
         "아이구", "아이쿠", "아이고",'라고','지난','당시','오전','오후',"어", "나", "우리", "저희", "따라", "의해",'데일리',
         '이날','갑자기','같다','는데','다른','에서','하는','하게','이후','통해','이날','정말','다시','스스로','같은','지난주',
         '이번주','다음주','그런','이런','첫째','둘째','셋째','넷째','다섯째','여섯째','일곱째','여덟째','아홉째','열째','기성',
             '이었','라는','저번','이번','면서','이렇게','저렇게','그렇게','정근','판청','아르테',
            '앵커','아나운서','좌윤','주고', 'B씨','A씨','경우','서로','대로','지사','인근','스완','위해','대한',
            '통한','기성','영제','간다','대해','수준','젊은','스탠','이씨','헤아','하라','패스트','좌안','이토','2TV',
            '성은','산다','나요','선우','누나','억원','태수','그냥','사이','어마','위엄','남자','로서','우리','누리','관련',
            '윤종','다친','승헌','골망','있는','까지','반종빈','녹이','현재','세이','카스티','스티','물론','라덴','상대로',
            '여러','아미','아사다','이닝','유기','신서','티브이데일리','공동','윤이','치원','타고','박씨','루수','유력','타고',
            '하고','처음','김씨','연재','사람','다이어','이다','인근','니다','아직','이씨','탈린','윤계','이날',
            '이것','저것','그것','신잡','장영','고씨','박씨','사실','경정','손새','장의','미도','말했',
            '빈살','실제','분들','건너','최근','조씨','장씨','올해','내년','작년','지난해','여서','A군','B군',
            '고든','램지','진혁','심지어','D씨','최근','니다','협회','그리즈','미오','그것','메이','이재','윈터',
            '임오','레이먼','사는','잡고','이야','치치','있다','1호','2호','3호','4호','5호','6호','7호','8호','9호',
            '다양한','부분','심씨','트루시','트루','심씨','부분','전체','주씨','펀치','원투','A사','1차','2차','3차','4차',
            '5차','6차','7차','8차','9차','10차','계속','MK','매경닷컴','MK스포츠','티노','처음','모현','들이','있는',
            '더욱','차장','부장','시점','000','달라','모든','구름','맑음','방향','같아','믹스','나인','윤균','모두',
            '라며','헤아','제트','라카','소속','1분','신잡','울스','일단','번째','어서','이지','원장','교수','선수',
            '안젤리나','브래드','드려','조원','둥지','빅리그','어떻','대답했','루프','트리플','빅토르','지금','때문','해성',
             '빠졌','그리팅','5일','점프','그룹','시즌','크게','없는']
swords2 = ['중인' ,'만큼', '마찬가지', '꼬집었', "기자","아", "휴", "아이구",
                      "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "가"] + pressList['언론사'].tolist() + ["저", "것",
    "동시에", "몇", "고려하면", "관련이", "놀라다", "무엇", "어느쪽", "오", "정도의", "더구나", "아무도", "줄은모른다", "참", "아니",
         "휘익", "향하다", "응당", "알겠는가", "인젠", "그래서", "자신", "해서는", "둘", "이었다", "임에", "하도록시키다", "누구", "이때",
         "삼", "제외하고", "쿵", "하면", "좀", "그렇지않으면", "아니었다면", "이라면", "팍", "일", "통하여", "무엇때문에", 
         "보아", "하게하다", "하는", "이르다", "타다", "까지도", "오직", "도달하다", "잠깐", "외에", "심지어", "하려고하다", "게다가",
         "후", "알", "비하면", "헉헉", "근거로", "월", "따라서", "않는다면", "일지라도", "함께", "이유는", "흥", "혼자", "관하여", 
         "붕붕", "하다", "진짜로", "의해", "바와같이", "대하면", "퍽", "보다더", "그렇게", "끼익", "댕그", "시초에", "당장", 
         "하는것만", "누가", "만이", "만일", "이지만", "하마터면", "꽈당", "만은", "우선", "없다", "휴", "하도록하다", "그런데",
         "비로소", "하게될것이다", "만큼 어찌됏든", "오히려", "을", "더라도", "안", "왜냐하면", "습니다", "줄은", "그리하여", "하",
         "어떻게", "대로", "기대여", "끙끙", "예를", "와르르", "이리하여", "이", "조차", "하고", "이젠", "뒤이어", "할줄알다", "반대로",
         "시각", "펄렁", "잇따라", "공동으로", "비록", "가까스로", "여덟", "비슷하다", "이상", "차라리", "이어서", "모두", "툭", "조차도",
         "헉", "부터", "혹시", "않고", "우리", "삐걱", "여보시오", "허", "해요", "견지에서", "하기는한데", "토하다", "않으면", "이봐", "관계가", 
         "한다면", "시작하여", "연이서", "이외에도", "그", "운운", "에게", "그럼에도", "예", "만약에", "했어요", "결과에", "제", "오자마자",
         "것들", "약간", "것과", "일때", "셋", "각종", "아이구", "같은", "향해서", "일것이다", "해야한다", "아이야", "로", "편이", "등등", 
         "해도좋다", "하기에", "김에", "몰랏다", "같이", "하도다", "즉시", "갖고말하자면", "우에", "어느", "허허", "하자마자", "에서", "그래도",
         "하여야", "된이상", "까악", "한켠으로는", "많은", "그중에서", "사", "낼", "뿐만", "저쪽", "어쩔수", "어떤것들", "물론", "결론을", "이만큼",
         "이렇게되면", "소인", "바꾸어말하면", "들", "이렇구나", "하물며", "얼마간", "얼마든지", "한항목", "하는것도", "졸졸", "한마디", "말할것도", 
         "만약", "남들", "총적으로", "허걱", "그리고", "따지지", "구체적으로", "못하다    하기보다는", "언제", "따르는", "구토하다", "앞에서",
         "대해서", "아", "앞의것", "비걱거리다", "헐떡헐떡", "어찌하든지", "입장에서", "의", "마저", "바로", "하기만", "않기", "또한", "쓰여",
         "위해서", "의거하여", "인", "아니면", "를", "사람들", "할수있다", "일곱", "근거하여", "한적이있다", "함으로써", "낫다", "어떤것",
         "방면으로", "중의하나", "어", "무릎쓰고", "저것만큼", "서술한바와같이", "그런즉", "들자면", "하지", "아이고", "불문하고", "만", 
         "마저도", "얼마만큼", "예컨대", "이렇게말하자면", "연관되다", "않다면", "들면", "이쪽", "의지하여", "여섯", "그저", "아니다", 
         "그렇지만", "기준으로", "되어", "가", "무렵", "즉", "말하면", "어찌", "그럼", "그위에", "그런", "조금", "매번", "혹은",
         "이천구", "중에서", "따름이다", "하기", "가령", "잠시", "아무거나", "하기보다는", "주저하지", "당신", "봐라", "그렇지",
         "자기집", "할지라도", "요만한걸", "우르르", "못하다", "왜", "이렇게", "퉤", "관계없이", "그래", "대해", "쪽으로", 
         "저것", "자기", "아홉", "지만", "구", "하지마", "따위", "하지만", "나", "해도", "전자", "그만이다", "안된다", "까닭으로", 
         "되다", "오르다", "딱", "다음에", "너희들", "점에서", "아이쿠", "쾅쾅", "종합한것과같이", "할수있어", "그치지", "비교적",
         "륙", "되는", "개의치않고", "엉엉", "하든지", "때가", "영차", "바꿔", "더불어", "주룩주룩", "따라", "이용하여", "우리들",
         "여기", "더욱이는", "하더라도", "입각하여", "여러분", "마치", "하느니", "너", "어디", "제각기", "밖에", "봐", "위하여", 
         "팔", "요만큼", "가서", "아니라면", "지든지", "참나", "할만하다", "타인", "든간에", "하겠는가", "거바", "겨우", "다음", 
         "이러한", "이럴정도로", "각자", "어때", "지말고", "형식으로", "그러한즉", "아니나다를가", "할", "불구하고", "지경이다",
    "어떠한", "기점으로", "할때", "등", "다시", "시키다", "답다", "소생", "라", "로써", "각", "부류의", "알았어", "훨씬", "위에서",
    "뿐이다", "시간", "그러나", "하곤하였다", "일단", "막론하고", "좋아", "솨", "이곳", "뿐만아니라", "아울러", "옆사람", "다수", 
    "예하면", "령", "어떤", "어떻해", "할수록", "말하자면", "전후", "메쓰겁다", "에", "으로써", "이번", "하면된다", "이것", "딩동",
"양자", "달려", "본대로", "탕탕", "마음대로", "쉿", "미치다", "다시말하면", "동안", "그러니까", "과연", "뚝뚝", "거의", "이천팔",
"이로", "않도록", "또", "한하다", "아래윗", "수", "다소", "어느것", "까지", "남짓", "저기", "관한", "무슨", "그에", "년도", 
"삐걱거리다", "이러이러하다", "와", "넷", "쳇", "논하지", "습니까", "이천육", "기타", "오로지", "어느곳", "설령", "할지언정", "칠",
"다만", "반드시", "한데", "곧", "의해서", "얼마나", "아니라", "상대적으로", "너희", "있다", "인하여", "다섯", "생각이다", "몰라도", 
"정도에", "버금", "까닭에", "얼마큼", "전부", "로부터", "힘입어", "틈타", "해도된다", "나머지는", "흐흐", "그때", "하여금", "모", 
"이런", "바꾸어서", "비추어", "각각", "설사", "이래", "비길수", "하지마라", "응", "다른", "듯하다", "보는데서", "어쨋든", "대하여", 
"좍좍", "으로", "여차", "틀림없다", "과", "고로", "요컨대", "일반적으로", "줄", "하는바", "그들", "요만한", "윙윙", "콸콸", "어기여차",
"언젠가", "이와", "할망정", "이천칠", "네", "없고", "둥둥", "겸사겸사", "그러므로", "안다", "거니와", "년", "여부", "때문에", "된바에야",
"향하여", "때", "하하", "및", "오호", "하면서", "더군다나", "한", "이유만으로", "어이", "하나", "저희", "더욱더", "두번째로", "바꾸어말하자면",
"이와같다면", "이르기까지", "단지", "그러면", "야", "결국", "영","뒤따라", "즈음하여", "도착하다", "와아", "다음으로", "같다", "자",
"아하", "생각한대로", "외에도", "의해되다", "설마", "으로서", "보면", "할뿐", "첫번째로", "아야", "어째서", "하는것이", "하구나", "않다",
"힘이", "육", "그러니", "여전히", "어찌됏어", "어찌하여", "어느해", "앗", "게우다","보드득", "관해서는", "자마자", "매", "하고있었다", 
"어느때", "여", "실로", "해봐요", "얼마", "아이"]

In [16]:

class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ot
        self.ctwitter = ct
        self.mecab = mecab
        
        #self.stopwords = []
        self.stopwords = swords +swords2 + stopwordList
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                #nouns.append(' '.join([noun for noun in self.ctwitter.phrases(str(sentence))
                #                                    if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.ctwitter.nouns(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.ctwitter.morphs(str(sentence))
                #                                   if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.mecab.nouns(str(sentence))
                                                  if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.mecab.morphs(str(sentence))
                                                   if noun not in self.stopwords and len(noun) > 1]))
        #nouns = list(set(nouns))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer(ngram_range=(1,4),sublinear_tf=True, lowercase=False, min_df=0.0001)
        self.cnt_vec = CountVectorizer(ngram_range=(1,4), lowercase=False,min_df=0.0001,)
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        #tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRankX(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

# 분석

* 중복되서 나오는 것 제거
* 혼자 나왔는데 태그가 이상한거 제거

In [17]:
extract_keywords_dict = dict()
#site = 'Naver'
site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/daum2/'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/naver2/'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)
keywordList = list()
for data in useCollection.find({'site':site}):
    #if len(data['mainText'].strip()) == 0:
        #print (data['link'])
        #break
    idIs = data['_id']._ObjectId__id
    tis2 = html.unescape(data['title'])+'. '+html.unescape(data['mainText'])
    #tis2 = '.\n'.join(tis.split('. '))
    #tis2 = data['mainText']
    if data['keywords'] !='NaN':
        keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
    else:
        pass
    print (data['link'])
    print (data['title'])
    print (data['keywords'])
    try:
        workrank = TextRankX(tis2)
    except:
        try:
            workrank = TextRankX(tis2)
        except:
            workrank = TextRankX(tis2)
            break
    extract_keywords_dict[idIs.hex()]= workrank.keywords()
    print (workrank.keywords())
    #print (lr)
    print ()
    
    '''
    if not os.path.isfile(etri_outcome+idIs.hex()+'.picked.txt'):
        try:
            etri = USE_ETRI_ANALYSIS('srl', tis2)
        except:
            print (etri[1])
            break
        else:
            pickle.dump(etri[0], open(etri_outcome+idIs.hex()+'.picked.txt','wb'))
            '''



http://v.media.daum.net/v/20171205151743503
김국진·노홍철·한혜진 '한명회', 오늘(5일) 9회만에 종영
['노홍철', '한명회', '한혜진', '종영', '편성시간']
['이름', '한명회', '방송', '출연자', '특집', '녹화', '출연', '출연 이름', '내린다', '한명회 내린다']

http://v.media.daum.net/v/20171209104014068
'몸 만드는 게 우선' 류중일 감독, 캠프 명단에 신인 없다
['류중일']
['캠프', '감독', '류중일 감독', '류중일', '캠프 명단', '오키나와 캠프', '오키나와', '신인', '일정', '명단']

http://v.media.daum.net/v/20171205061623329
[어제TV]첫방 '저글러스' 男보스 女비서, 시대착오적 설정 '불편'
['불륜', '비', '첫방', '최다니엘', '최대철']
['비서', '상무', '보스', '봉상', '수습', '불편', '설정', '봉상 아내', '아내', '방송']

http://v.media.daum.net/v/20171205140114189
[Oh! 무비] '기억의 밤' 장항준 감독은 어떻게 골리앗을 무찔렀나
['기억의밤', '시나리오', '리얼리티', '강하늘', '김무열']
['감독', '기억', '스릴러', '장항준 감독', '장항준', '영화', '드라마', '시나리오', '현실', '스릴']

http://v.media.daum.net/v/20171205065011570
[RE:TV]'섬총사' 강호동X정상훈, 80cm 대어 낚아 '낚시TV 따로없네'
['강호동', '정상훈', '조세호', '어청도', '달타냥']
['정상훈', '강호동', '조세호', '어청도', '강호동 정상훈', '낚시', '섬총사', '멤버', '농어', '도착']

http://v.media.daum.net/v/20171204154759778
'범죄도시'는 왜 불법 다운과의 전쟁을 선포했나
['윤계상', 

['동탁', '수창', '동탁 수창', '지안', '수사', '여고생', '투깝스', '박실장', '마음', '납치']

http://v.media.daum.net/v/20171203125355093
포체티노, "나은 미래를 위한 고통"
['포체티노']
['토트넘', '포체티노', '감독', '경기', '포체티노 감독', '이야기', '미래', '무승부', '손흥민', '손흥민 동점골']

http://v.media.daum.net/v/20171203112104835
'영흥도 낚싯배 전복' 짧은 시간에 인명피해 컸던 이유
['이슈 · 영흥도 낚싯배 전복사고', '낚싯배', '영흥도', '급유선']
['인터뷰', '사고', '구조', '해서', '낚싯배', '가능', '발생', '해역', '충돌', '주변']

http://v.media.daum.net/v/20171205165305779
[국제] 중국, '하얀 석유' 리튬 싹쓸이?
['리튬', '전기차', '배터리']
['리튬', '중국', '확보', '배터리', '세계', '전기차', '최대', '리튬 확보', '개발', '호주']

http://v.media.daum.net/v/20171206140301737
[해보니 시리즈 19] '집 값 지옥 교통 지옥' 주말 부부의 현실
['대중교통', '빌라', '월세']
['부부', '주말', '남편', '주말 부부', '서울', '상암', '빌라', '거리', '출퇴근', '40']

http://v.media.daum.net/v/20171203122805860
'두산맨' 최대성, "몸 상태 좋아..팀에 도움 되겠다"
NaN
['최대', '상태', '많이', '각오', '실시', '항상', '참석', '더라', '모습', '보여']

http://v.media.daum.net/v/20171208102101715
"사장이 저를 '개XX'라 부르지만, 버티고 있어요"
['갑질', '공유', '폭언']
['갑질', '직장', '피해자', '직장 갑질', '가면', '회

['아침', '서해', '영향', '제주도', '서울', '내일', '전국', '기상청', '남하', '북서쪽']

http://v.media.daum.net/v/20171205065021584
[RE:TV]"잘 하려고 애쓰지 마"..'동상이몽2' 장신영 향한 따뜻한 위로
['강경준', '시어머니', '신승훈', '우효광', '동상이몽']
['장신영', '시어머니', '강경준', '방송', '김장', '동상이몽', '출연', '신승훈', '응원', '연습']

http://v.media.daum.net/v/20171209160144985
[사진]김예령,'야구선수 윤석민이 제 사위예요~'
['김수현', '결혼', '윤석민', '결혼식']
['김수현', '윤석민', '결혼식', '윤석민 김수현', '김예령', '결혼 예정', '아들', '아들 출산 결혼', '결혼', '아들 출산']

http://v.media.daum.net/v/20171203103735000
"세월호 때와는 달랐다" 文대통령, 낚싯배 전복사고 긴급 진두지휘(종합)
['이슈 · 영흥도 낚싯배 전복사고', '낚싯배', '전복사고', '세월호']
['지시', '구조', '대통령', '현장', '파악', '필요', '낚싯배', '사고', '행안부', '장관']

http://v.media.daum.net/v/20171203190258781
며느리 성폭행 70대에 징역 7년 '중형'
['며느리', '성폭력범죄', '낙태']
['며느리', '성폭행', '성폭력', '중형', '범행', '아들', '재판부', '밝혔', '선고', '도록']

http://v.media.daum.net/v/20171203100441479
헥터급 아닌 린드블럼의 행선지는 어차피 롯데?
['조쉬 린드블럼']
['린드블럼', '롯데', '구단', '영입', '달러', '외국인', '투수', '보류', '연봉', '달러 연봉']

http://v.media.daum.net/v/20171201060029695
황재균 영입을 가장 반

['핀란드', '속초', '막걸리', '한국', '바다', '3인방', '만족', '만점', '표현', '페트리']

http://v.media.daum.net/v/20171206073112505
평창 '바가지 올림픽' 역풍..숙박업소 대규모 공실 우려
['숙박업소', '바가지', '평창']
['올림픽', '예약', '관람객', '강원도', '업소', '계약', '가격', '숙박업소', '강릉', '기준']

http://v.media.daum.net/v/20171201193331290
[단독] SBS 정주희 기상캐스터, 4년 열애 끝 '품절녀' 됐다
['신혼여행', '결혼식']
['정주희', '기상캐스터', '정주희 기상캐스터', '축하', '날씨', '날씨 뉴스', '뉴스', '축하 부부', '부부', '뉴스 날씨 뉴스']

http://v.media.daum.net/v/20171203122651845
[박은별의 도미니카 Live] 로사리오, 잇따른 러브콜에도 윈터리그 뛰는 이유
['윌린 로사리오']
['로사리오', '한국', '도미니카', '한화', '리그', '일본', '도미니카 리그', '한신', '계약', '엠스플']

http://v.media.daum.net/v/20171201171509864
[단독] '의리남' 김생민, 불량 에이전시 퇴출에 동참..CF 거액 거절
['미지급', '눈길', '김우빈', 'sm', '고수']
['불량', '김생민', '광고', '협업', '협업 금지', '금지', '상벌위', '모델료', '광고 모델료', '거절']

http://v.media.daum.net/v/20171209193005504
40대에 폐경.. 왜 이리 억울하고 서럽지
['이리', '출산', '임신']
['의사', '폐경', '여성', '아무', '수치', '호르몬 수치', '호르몬', '뉴스', '머릿속', '개인']

http://v.media.daum.net/v/20171211133422561
[원투펀치 287회 2부] 우리는 월드컵에서 어

['황금빛', '소현경', '황금빛 인생', '인생', '작가', '소현경 작가', '시청', '서영', '기록', '드라마']

http://v.media.daum.net/v/20171201152304366
'33년 절친' 우병우-최윤수, 나란히 구속 위기
['이슈 · 국정원·군 정치개입 의혹', '우병우', '최윤수', '구속영장']
['수석', '구속', '검찰', '영장', '구속 영장', '조사', '구속 영장 청구', '영장 청구', '청구', '검사']

http://v.media.daum.net/v/20171201115400610
국방부 "JSA 북한군 13초 사격..귀순자 낙엽 덮어 은폐시도"
['이슈 · JSA 초소 북한군 귀순', '북한군', '사격', '귀순자']
['국방부', '귀순자', '사격', '북한', '확인', '시도', 'MDL', '은폐', '설명', '월경']

http://v.media.daum.net/v/20171206013300137
분노에 휩싸인 '자중지란' 한국당..여당 도와줬다?
['이슈 · 2018 예산안 본회의 통과', '한국당', '본회의', '의원총회']
['한국당', '의원', '입장', '본회의장', '의원 총회', '총회', '반대', '본회의', '회의장', '모습']

http://v.media.daum.net/v/20171206072906455
김문호-나경민-이병규 경쟁, '국대급' 롯데 외야는 전쟁터
['김문호', '이병규', '나경민']
['손아섭', '롯데', '민병헌', '외야', '김문호', '이병규', '경쟁', '이대호', '지명타자', '주전']

http://v.media.daum.net/v/20171201085119265
밀란의 중국 자본, 알고 보니 막대한 '빚'
NaN
['밀란', '리용', '투자자', 'AC 밀란', 'AC', '투자', '자본', '가능', '전문가', '분석']

http://v.media.daum.net/v/20171203083011370
[시나쿨파

...

['신과함께', '작품', '하정우', '감독', '미스터', '관객', '이야기', '개봉', '김용화 감독', '김용화']

http://v.media.daum.net/v/20171226103907297
태민, 'KBS 가요대축제' 불참 확정.."휴식·안정 필요"[공식입장]
['kbs가요대축제', '휴식', '더유닛', '황치열', '샤이니']
['가요대축제', '태민', '2017 가요대축제', '2017', '무대', '준비', '다는', '상황', '샤이니', '불참']

http://v.media.daum.net/v/20171226065029907
[SC리뷰] "월세만 800만원"..'비행소녀' 이태임, 무너진 '한강뷰' 로망
['이태임', '한강', '비혼', '드림하우스', '조윤선']
['이태임', '한강', '어머니', '중개업자', '부동산 중개업자', '현실', '비행소녀 이태임', '비행소녀', '월세', '부동산']

http://v.media.daum.net/v/20171226112005077
[엑's 스타] "떡잎부터 남달랐다"..슈퍼루키들의 '더 풋풋한' 데뷔시절
['신혜선', '양세종', '우도환', '채수빈', '황금빛내인생']
['드라마', '연기', '양세종', '주인공', '우도환', '채수빈', '인생', '캐릭터', '신혜선', '시청자']

http://v.media.daum.net/v/20171226171356245
"아빠 딸 같죠" 이연복X전소미, 세대 뛰어넘은 우정
['이연복', '냉장고를부탁해', '눈길', '메리크리스마스', '크리스마스']
['이연복', '전소미', '셰프', '이연복 셰프', '사진', '사진 이연복', '이우인', '인스타그램', '게재', '아빠']

http://v.media.daum.net/v/20171226210718587
김구라 "故 종현, 30~40대가 더 멋지겠다 생각했는데"
['김구라', '샤이니', '아이돌', '청담동']
['종현', '김구라', '본격 연예 한밤', 

['최대', '투수', '감독', '150', '야구', 'km', '150 km', '롯데', '입단', '기록']

http://v.media.daum.net/v/20171226060029396
[IS 포커스] '장수 외인' 니퍼트·해커, 왜 인기가 없나
['더스틴 니퍼트']
['니퍼트', '해커', '외국인', '달러', '연봉', '두산', '리그', '투수', '구단', 'NC']

http://v.media.daum.net/v/20171226052909214
울산 이근호 영입 철회, 강원 20억 요구
['이근호']
['강원', '이적료', '이근호', '울산', '리그', '영입', '이적', '연봉', '이근호 영입', '모양새']

http://v.media.daum.net/v/20171226112717343
배드민턴 '천재소녀' 등장..성인 선수들 연파 여중생 사상 첫 태극마크
NaN
['안세영', '대표', '감독', '배드민턴', '여자단식', '선발전', '국가 대표', '국가', '성인', '태극마크']

http://v.media.daum.net/v/20171226142701660
[EPL 현미경] '9호골 도전' 손흥민, 일본 자존심 요시다 마야 뚫을까
NaN
['손흥민', '주전', '요시다', 'EPL', '선발', '승점', '영국', '힘겨운', '수비수', '포함']

http://v.media.daum.net/v/20171226173940945
'토트넘 MF' 완야마, 가장 웃긴 동료로 손흥민 지목
['손흥민', '빅토르 완야마']
['손흥민', '완야마', '토트넘', '동료', '가장', '알리', '케인', '26', '25', '완야마 가장']

http://v.media.daum.net/v/20171226010318268
'이적설' 기성용, 웨스트햄이 스완지보다 나은 이유 셋
['기성용', '스완지 시티 AFC']
['웨스트햄', '스완지', '런던', '감독', '안정', '보다', '기성용', '이적', '유나

['구호', '활동', '경찰', '화재', '사우나', '구호 활동', '건물', '점유자', '판단', '직원']

http://v.media.daum.net/v/20171227044205437
영화 보는데 뒤에서 의자 '툭툭'.. 관크족(族)을 아시나요?
['직장인', '스마트폰', '카톡']
['관객', '피해', '직장인', '영화', '공연', '관람', '스마트폰', '제대로', '남성', '정도']

http://v.media.daum.net/v/20171227071806666
홍준표, 후보 영입 초반부터 '빨간불'..잇단 손사래
['이슈 · 2018년 6·13 지방선거', '홍준표', '지방선거', '한국당']
['대표', '후보', '선거', '지방', '지방 선거', '시장', '한국당', '계획', '대법관', '다는']

http://v.media.daum.net/v/20171227162235895
한국, 中 네티즌이 뽑은 비우호 국가 5위..1위 호주
['호주', '사드', '인도']
['중국', '여론', '차지', '호주', '한국', '악화', '결과', '미국', '우호', '한중']

http://v.media.daum.net/v/20171227072727760
"죽더라도 건물 뛰어들걸.." 죄책감에 괴로운 소방관들
['이슈 · 제천 스포츠센터 화재 참사', '화재', '소방관', '스포츠센터']
['구조', '건물', '소방관', '피해', '김용빈', '화재', '나섰', '구조 나섰', 'News 김용빈', 'News']

http://v.media.daum.net/v/20171227213213032
'청산가리 6천배' 협죽도, 학교 앞 산책로 도처에 깔려
['부산시', '이상은', '구토']
['협죽도', '해운대구', '성분', '청산가리', '부산', '산책로', '부산 해운대구', '학교', '나무', '그루']

http://v.media.daum.net/v/20171227000221030
개인소득 1위 10년 

['송민호', '멤버', '나영석', '나영석 PD', 'PD', '강식당', '주방', '강식당 나영석', '은지원', '웃음']

http://v.media.daum.net/v/20171227133035809
[단독] KBS 박소현 아나♥커피소년 열애ing..11살 나이차
['박소현', '싱어송라이터', '연인사이', '음악', '나레이션']
['박소현', '커피소년', '사랑', '연인', '진행', '김수정', '키워', '열애', '2015', '11']

http://v.media.daum.net/v/20171227164710682
[단독] 결국 '화유기' 긴급수혈..'구가의서' 김정현 감독 투입
['김정현', '방송사고', '고대소설', '서유기', '차승원']
['화유기', '사고', '방송', '감독', '김정현 감독', '김정현', '된다', '연출', '방송 사고', '투입']

http://v.media.daum.net/v/20171227065029300
[어저께TV]'불청' 권선국, 이토록 화끈한 결혼 고백 "혼인신고 했다"
['권선국', '혼인신고', '결혼', '여자친구', '김국진']
['권선국', '결혼', '여자', '아버지', '고백', '불청', '결혼 고백', '전했', '다는', '친구']

http://v.media.daum.net/v/20171227065028299
[어저께TV] '뭉쳐야뜬다' 알베르토, 고정이 시급합니다
['알베르토', '뭉쳐야뜬다', '다니엘', '비정상회담', '패키지여행']
['멤버', '알베르토', '뭉쳐야 뜬다', '뜬다', '뭉쳐야', '한국', '다니엘', '모습', '외국인', '외국인 친구']

http://v.media.daum.net/v/20171227105125628
'강식당', 자기복제 우려 뚫고 어마어마한 시청률 낸 비법
['이수근', '오므라이스', '송민호', '우리들', '시청률']
['강식당', '반복', '상황', '웃음', '이수근', '된다', '일상', '장면'

['케인', '기록', '시어러', '손흥민', '경기', '생각', '2017', '최고', '최다골', '해트트릭']

http://v.media.daum.net/v/20171227053830693
손흥민 향한 외신의 극찬 "SON은 선물, 골은 당연"
['손흥민', '토트넘 핫스퍼 FC']
['손흥민', '케인', '도움', '후반', '극찬', '2017', '기록', '사우스햄튼', '1골', '1골 도움']

http://v.media.daum.net/v/20171227110740363
그림책 뚫고나온듯, 아름답고 따뜻하여라..'평창 시상패션'
NaN
['시상식', '시상', '한국', '평창', '올림픽', '패럴림픽', '시상품', '전통', '메달 시상식', '시상대']

http://v.media.daum.net/v/20171227010121388
[GOAL 현장인터뷰] '신기록' 케인 "손흥민, 좋은 친구이자 뛰어난 선수"
['손흥민', '토트넘 핫스퍼 FC', '해리 케인']
['케인', '손흥민', '현지', '기록', '해트트릭', '해트트릭 기록', '오늘', '사우스햄튼', '해리 케인', '해리']

http://v.media.daum.net/v/20171227164442606
美 매체 '김현수 트레이드는 실패작'
NaN
['김현수', '볼티모어', '디드', '계약', '헬릭슨', '트레이드', '한국', '못했', '김현수 메이저리그', '메이저리그']

http://v.media.daum.net/v/20171227133044813
두산, FA 김현수 보상선수로 투수 유재유 지명
['김현수']
['재유', '두산', '지명', '1라운드', 'LG', '출신', '김현수', '투수 재유', '투수', '김현수 보상']

http://v.media.daum.net/v/20171227103758132
KIA 유력? '대기록 앞둔' 정성훈의 예상 행선지는
['정성훈']
['정성훈', 'KIA', 'LG', '롯데', '1루수', '

In [20]:
pickle.dump(extract_keywords_dict, open('./data/Keyword_outcome/mecab-daum.pickled','wb'))

* 형태소 분석 : morp
* 어휘의미 분석 : wsd (동음이의어)
* 어휘의미 분석 : wsd_poly (다의어)
* 개체명 인식 : ner
* 의존구문 분석 : dependency
* 의미역 인식 : srl

# sentiment analysis